# Aggregation

## Lecture objectives

1. Demonstrate how to aggregate in `pandas` using `groupby`

Let's start off by loading the data that we used in the previous lecture, and joining two of the dataframes.

In [ ]:
import pandas as pd
collisionDf = pd.read_csv('../data/Collisions.csv')
partyDf = pd.read_csv('../data/Parties.csv')
victimDf = pd.read_csv('../data/Victims.csv')
collisionDf.set_index('CASE_ID', inplace=True)
partyDf.set_index('CASE_ID', inplace=True)
joinedDf = collisionDf.join(partyDf, rsuffix='_from_party')
joinedDf.head()

Many of the questions that we want to ask involve some form of aggregation, that creates a useful summary. And for this, `groupby` is the key. Basically, think what you want to aggregate over, and that's the field that you group by.

Example: what's the mean of killed and injured people by intoxication status? (Consult the [codebook](https://tims.berkeley.edu/help/SWITRS.php#Party_Level), sometimes called a data dictionary, for an explanation of what each code means.)

In [ ]:
joinedDf.groupby('PARTY_SOBRIETY')[['NUMBER_KILLED','NUMBER_INJURED']].mean()

<div class="alert alert-block alert-info">
<strong>Warning:</strong> Are these accurate averages per collision? What might we need to do (conceptually)?
</div>

We have the correct mean for the average over parties. 

But remember that we did a one-to-many join earlier on. Some collisions are therefore duplicated in `joinedDf`, so the simple mean in effect weights collisions by the number of parties.

Sometimes, we need to transform the data first. Let's look at collisions that involve pedestrians.

In [ ]:
print(joinedDf.groupby('PARTY_SOBRIETY').PEDESTRIAN_ACCIDENT.mean())

<div class="alert alert-block alert-info">
    <strong>Exercise:</strong> What went wrong? <em>Hint:</em> the first step is to look at the data field.
</div>

In [ ]:
print(joinedDf.PEDESTRIAN_ACCIDENT.head())
print(joinedDf.PEDESTRIAN_ACCIDENT.unique())

It looks like we are trying to calculate the mean of a column that has `Y` and `NaN` values. `pandas` doesn't know how to do that.

We can fix this error by creating a new column, `ped_accident_boolean`, which is `True` if the value is `Y` and `False` otherwise. Then we can take the mean of the boolean column (`True` is considered a `1` and `False` a `0`).

In [ ]:
joinedDf['ped_accident_boolean'] = joinedDf.PEDESTRIAN_ACCIDENT=='Y'
# let's look at the results
joinedDf[['PEDESTRIAN_ACCIDENT','ped_accident_boolean']].head()

This function would do exactly the same, and is easier to adapt if there are multiple values (e.g. `Y`, `N`, or missing) - you can just add more `elif` statements.

In [ ]:
def convert_to_bool(ped_accident):
    if ped_accident=='Y':
        return True
    else:
        return False
joinedDf['ped_accident_boolean'] = joinedDf.PEDESTRIAN_ACCIDENT.apply(convert_to_bool)
joinedDf[['PEDESTRIAN_ACCIDENT','ped_accident_boolean']].head()

A more elegant and concise way is to use a `lambda` (anonymous) function.

Here, the value for `PEDESTRIAN_ACCIDENT` for that row is passed to the function as the variable `x`. Then that function returns `1` if x is `Y`, otherwise `0`.

In [ ]:
joinedDf['ped_accident_boolean'] = joinedDf.PEDESTRIAN_ACCIDENT.apply(
                                        lambda x: True if x=='Y' else False)

joinedDf[['PEDESTRIAN_ACCIDENT','ped_accident_boolean']].head()

Now we can look at the results using `groupby`. For parties of different sobriety states, what are the proportions of pedestrian accidents? 

In [ ]:
print(joinedDf.groupby('PARTY_SOBRIETY').ped_accident_boolean.mean())

It's not just means that you can generate with `groupby`. Standard deviations, counts, sums, and more are available. [See the documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) for more examples of this very powerful function.

You can also group by the index, rather than a column, using `level=0`. This is useful if (say) your observations are indexed by census tract, and you want to calculate tract-level means or sums.

We can set `PARTY_SOBRIETY` as the index to illustrate this.

In [ ]:
# size() gives the number of rows in each group
print(joinedDf.groupby('PARTY_SOBRIETY').ped_accident_boolean.size())

# identical results if we set the index to PARTY_SOBRIETY and group by the index
tmpDf = joinedDf.set_index('PARTY_SOBRIETY')
print(tmpDf.groupby(level=0).ped_accident_boolean.size())

# or if we do the above in a single line
print(joinedDf.set_index('PARTY_SOBRIETY').groupby(level=0).ped_accident_boolean.size())

In [ ]:
# sum gives the group-wise totals, i.e. the number of True values
print(joinedDf.groupby('PARTY_SOBRIETY').ped_accident_boolean.sum())

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Group-wise means, totals, and sums are easy to compute using groupby.</li>
</ul>
</div>